In [1]:
import sys
sys.path.append('../')

In [2]:
import arcpy
import os
import zipfile
import shutil

In [3]:
from scripts.utils import init_gdb, og_file_input
workspace, scratch_workspace = init_gdb()

In [4]:
year = str(2023)

In [5]:
ITS_pts = og_file_input(prefix = 'Treat_n_harvests_points'+year, filetype = 'Point', gdb = os.path.join(workspace, 'D_Appended'))
pts_path = os.path.join(workspace,'D_Appended', ITS_pts)
ITS_lns =  og_file_input(prefix = 'Treat_n_harvests_lines'+year, filetype = 'POLYLINE', gdb = os.path.join(workspace, 'D_Appended'))
lns_path = os.path.join(workspace,'D_Appended',ITS_lns)
ITS_ply = og_file_input(prefix = 'Treat_n_harvests_polygons'+year, filetype = 'POLYGON', gdb = os.path.join(workspace, 'D_Appended'))
ply_path = os.path.join(workspace,'D_Appended',ITS_ply)

In [6]:
act_rep = og_file_input(prefix = 'Activities_Report', filetype = 'Point', gdb = os.path.join(workspace, 'f_Report_Data'))
ar_path = os.path.join(workspace,'f_Report_Data', act_rep)
ftp_rep =  og_file_input(prefix = 'Footprint_Report', filetype = 'POLYGON', gdb = os.path.join(workspace, 'f_Report_Data'))
fr_path = os.path.join(workspace,'f_Report_Data',ftp_rep)

ftp_pts =  og_file_input(prefix = 'Footprint_pts', filetype = 'Point', gdb = os.path.join(workspace, 'f_Report_Data'))
fp_path = os.path.join(workspace,'f_Report_Data',ftp_pts)

In [7]:
gdb_path = []

In [8]:
### iterate through upload layers
### create .gdb with specific layer for upload
### zip the geodatabase for upload

for f_name, f_path in zip([ITS_pts, ITS_lns, ITS_ply, act_rep, ftp_rep, ftp_pts], [pts_path, lns_path, ply_path, ar_path, fr_path, fp_path]):
    # create geodatabase
    out_path = arcpy.CreateFileGDB_management(os.path.join(os.path.dirname(workspace), "ITSGDB_backup"), f_name)[0]
    gdb_path.append(out_path)
    fc = arcpy.management.CreateFeatureclass(
            out_path=out_path, 
            out_name=f_name,
            geometry_type=arcpy.Describe(f_path).shapeType,
            template=[f_path], 
            spatial_reference=arcpy.Describe(f_path).spatialReference.name
        )

    arcpy.management.Append(
        inputs=[f_path],
        target=fc,
        schema_type="NO_TEST",
        )
    # create zip file
    myzipfile = zipfile.ZipFile(out_path + ".zip", 'w', zipfile.ZIP_DEFLATED)

    for dirname, subdirs, files in os.walk(out_path):
        for f in files:           
            myzipfile.write(os.path.join(dirname, f), arcname = os.path.join(os.path.basename(dirname), f))


    
    myzipfile.close()



 

In [9]:
### DELETE .gdb after ZIP
for p in gdb_path:
    try:
        shutil.rmtree(p)
    except:
        print('file path: ', p, ' does not exist')
        continue

In [22]:
import arcgis
from arcgis.gis import GIS

gis = arcgis.gis.GIS("https://portal.sparcal.sdsc.edu/arcgis", "johnnylei")

Enter password:  ········


In [27]:
import time


In [26]:
for p in gdb_path:
    fgdb = p + ".zip"
    serviceProp = {}
    serviceProp['type'] = 'File Geodatabase'
    serviceProp['itemType'] = "file"
    serviceProp['tags'] = "ITS"

    pubProps = {}
    pubProps["name"] =os.path.basename(p) + "_test"


    fgdb1 = gis.content.add(item_properties=serviceProp, data = fgdb)
    fgdb2 = fgdb1.publish(publish_parameters = pubProps, file_type = 'filegeodatabase', overwrite=True)
    
    
    time.sleep(30)
    

Exception: Job failed.